This notebook looks at the networking needs for the transatlantic link and the US Tier-1 plus Tier-2 facilities (assuming the current mix of facilities). This thinking can be generalized, but in this notebook there are some national specific concepts programmed in.

Known problems

Transatlantic: Missing shutdown year model; Missing processing versions

In [22]:
from __future__ import print_function
def transatlantic_network_model(model,dataProduced):
    YEARS=dataProduced.keys()
    YEARS.sort()
    TYPES=[]
    TIERS=[]
    for year in YEARS:
        types_of_data = dataProduced[year].keys()
        for t in types_of_data: 
            if t not in TYPES:
                TYPES.append(t)
            for tier in dataProduced[year][t]:
                if tier not in TIERS:
                    TIERS.append(tier)

    PETA=1.e15
    transatlantic={}
    ta_model = model['transatlantic_network']
    for year in YEARS:
        transatlantic[year]=0.
        for tier in TIERS:
            for t in TYPES:
                if t in dataProduced[year] and tier in dataProduced[year][t]:
                    if tier in ta_model:
                        transatlantic[year]+=dataProduced[year][t][tier]*ta_model[tier]
                    else:
                        transatlantic[year]+=dataProduced[year][t][tier]*ta_model['default']
                        
    print("Network needs per year (PB)")
    for year in YEARS:
        str = '%5d %5d' % (year,transatlantic[year]/PETA)
        print (str)
    
    return


            


In [23]:
def network_model_tier1(model,dataProduced):
    
    #inbound traffic
    # transfers for T1 tape and disk from sites
    #outbound traffic
    # transfers to T2
    # remote read for premixing
    # remote read for analysis
    return


In [ ]:
def network_model_tier2(model,dataProduced):
    
    #inbound traffic
    # transfers for T2 disk from sites
    #outbound traffic
    # transfers to T1 for data/mc produced
    # remote read for analysis
    
    return



In [24]:
%run notebook_importer.ipynb
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = 10, 6

from resource_runner import run_computing_model
import resource_cpu
import CostEvolution

jsons=['json/RelyOnMiniAOD.json', 'json/Analysis.json', 'json/2018changes.json', 'json/IntroduceNanoAOD.json'] 
model=resource_cpu.create_model(jsons )
model['names']=jsons
cost_models=CostEvolution.define_cost_model()
model['cost_model']=cost_models

metrics,details=run_computing_model(model)
dataProduced=details['dataProduced']

#how much data from each data tier will we transfer across the atlantic
#this ignores the concept of prompt reco being done at CERN...
model['transatlantic_network']= { 'default' : 1.0-0.4, #all data/mc we don't produce in US
                                  'RAW' : 0.4, #US share of RAW data
                                  'USER' : 0.0 #no transfers of USER data
                                }

model['premixing'] = {'size_per_event' : 'hi there'}
model['national_tier_model'] = { 'nTier2' : 7}
transatlantic_network_model(model,dataProduced)
network_model_tier1(model,dataProduced)
network_model_tier2(model,dataProduced)


Network needs per year (PB)
 2016    13
 2017    13
 2018    13
 2019     9
 2020     9
 2021    19
 2022    19
 2023    19
 2024    11
 2025    43
 2026   202
 2027   499
 2028   499
 2029   499
 2030   199
